# SAMueL Outcomes

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import stroke_outcome as so

!pip install -U stroke-lifetime
# Function to import fixed params for either mRS or dicho model:
from stroke_lifetime.fixed_params import get_fixed_params
# Main function for calculating lifetime results:
from stroke_lifetime.main_calculations import main_calculations

In [2]:
data = pd.read_csv('../output/thrombolysis_choice_results.csv')
data = data.drop(columns=['Unnamed: 0'])

In [3]:
# Add columns
data['untreated_mrs_0-1'] = data['untreated_mrs_0'] + data['untreated_mrs_1']
data['untreated_mrs_0-2'] = data['untreated_mrs_0'] + data['untreated_mrs_1'] + data['untreated_mrs_2']
data['treated_mrs_0-1'] = data['treated_mrs_0'] + data['treated_mrs_1']
data['treated_mrs_0-2'] = data['treated_mrs_0'] + data['treated_mrs_1'] + data['treated_mrs_2']
data['untreated_mrs_5-6'] = data['untreated_mrs_5'] + data['untreated_mrs_6']
data['treated_mrs_5-6'] = data['treated_mrs_5'] + data['treated_mrs_6']

# Change in mRS 0-1
data['change_mrs_0-1'] = data['treated_mrs_0-1'] - data['untreated_mrs_0-1']
data['change_mrs_0-2'] = data['treated_mrs_0-2'] - data['untreated_mrs_0-2']
data['change_mrs_5-6'] = data['treated_mrs_5-6'] - data['untreated_mrs_5-6']

# Number neeeded to treat
data['nnt_mrs_0-1'] = 1 / data['change_mrs_0-1']
data['nnt_mrs_0-2'] = 1 / data['change_mrs_0-2']
data['nnt_mrs_5-6'] = 0 - (1 / data['change_mrs_5-6']) # Reverse sign as reduction is good

## Run analysis of outcomes on ischaemic stroke

In [4]:
analyse_infarction_only = True

mask = (data['infarction'] == 1)
if analyse_infarction_only:
    data = data[mask]
iscahaemic = mask.mean()
print(f'Proportion of patients with infarction: {iscahaemic:0.4f}')


Proportion of patients with infarction: 0.8540


In [5]:
patient_groups = dict()

In [6]:
proportion_treated = []

# Get actually treated population
mask = data['thrombolysis'] == 1
patient_groups['actually treated'] = data[mask]
treated = mask.mean()
proportion_treated.append(treated)
print(f'Proportion of patients treated: {treated:0.4f}')

# Get improved outcomes (including anticoagulant)
mask = data['improved_outcome'] == 1
patient_groups['improved_outcome including anticoagulant'] = data[mask]
improved_outcome = mask.mean()
proportion_treated.append(improved_outcome)
print(f'Proportion of patients with improved outcome (including use of anticoagulants): {improved_outcome:0.4f}')

# Get improved outcomes (excluding anticoagulant)
mask = (data['improved_outcome'] == 1) & (data['afib_anticoagulant'] == 0)
patient_groups['improved_outcome_no_anticoagulant'] = data[mask]
improved_outcome_no_anticoagulant = mask.mean()
proportion_treated.append(improved_outcome_no_anticoagulant)
print(f'Proportion of patients with improved outcome (excluding use of anticoagulants): {improved_outcome_no_anticoagulant:0.4f}')

# Get benchmark outcomes
mask = data['benchmark_decision'] == 1
patient_groups['benchmark'] = data[mask]
benchmark = mask.mean()
proportion_treated.append(benchmark)
print(f'Proportion of patients with benchmark outcome: {benchmark:0.4f}')

Proportion of patients treated: 0.3515
Proportion of patients with improved outcome (including use of anticoagulants): 0.4961
Proportion of patients with improved outcome (excluding use of anticoagulants): 0.4334
Proportion of patients with benchmark outcome: 0.4320


In [7]:
# Results dataframe
results = pd.DataFrame(index=patient_groups.keys())
results['proportion_ischaemic_treated'] = proportion_treated

# Put summary results in lists
untreated_mrs_0_1 = []
treated_mrs_0_1 = []
change_in_mrs_0_1 = []
untreated_mrs_0_2 = []
treated_mrs_0_2 = []
change_in_mrs_0_2 = []
untreated_mrs6 = []
treated_mrs6 = []
change_in_mrs6 = []
untreated_weighted_mrs = []
treated_weighted_mrs = []
change_in_weighted_mrs = []
untreated_utility = []
treated_utility = []
change_in_utility = []


for key, group in patient_groups.items():
    # Patients in MRS 0-1
    untreated = np.mean(group['untreated_mrs_0'] + group['untreated_mrs_1'])
    treated = np.mean(group['treated_mrs_0'] + group['treated_mrs_1'])
    improved = treated - untreated
    untreated_mrs_0_1.append(untreated)
    treated_mrs_0_1.append(treated)
    change_in_mrs_0_1.append(improved)
    # Patients in MRS 0-2
    untreated = np.mean(group['untreated_mrs_0'] + group['untreated_mrs_1'] + group['untreated_mrs_2'])
    treated = np.mean(group['treated_mrs_0'] + group['treated_mrs_1'] + group['treated_mrs_2'])
    improved = treated - untreated
    untreated_mrs_0_2.append(untreated)
    treated_mrs_0_2.append(treated)
    change_in_mrs_0_2.append(improved)
    # Patients in MRS 6 (dead)
    untreated = np.mean(group['untreated_mrs_6'])
    treated = np.mean(group['treated_mrs_6'])
    improved = treated - untreated
    untreated_mrs6.append(untreated)
    treated_mrs6.append(treated)
    change_in_mrs6.append(improved)
    # Weighted MRS
    untreated = np.mean(group['untreated_weighted_mrs'])
    treated = np.mean(group['treated_weighted_mrs'])
    improved = treated - untreated
    untreated_weighted_mrs.append(untreated)
    treated_weighted_mrs.append(treated)
    change_in_weighted_mrs.append(improved)
    # Utility
    untreated = np.mean(group['untreated_utility'])
    treated = np.mean(group['treated_utility'])
    improved = treated - untreated
    untreated_utility.append(untreated)
    treated_utility.append(treated)
    change_in_utility.append(improved)

results['untreated_mrs_0_1'] = untreated_mrs_0_1
results['treated_mrs_0_1'] = treated_mrs_0_1
results['change_in_mrs_0_1'] = change_in_mrs_0_1
results['untreated_mrs_0_2'] = untreated_mrs_0_2
results['treated_mrs_0_2'] = treated_mrs_0_2
results['change_in_mrs_0_2'] = change_in_mrs_0_2
results['untreated_mrs6'] = untreated_mrs6
results['treated_mrs6'] = treated_mrs6
results['change_in_mrs6'] = change_in_mrs6
results['untreated_weighted_mrs'] = untreated_weighted_mrs
results['treated_weighted_mrs'] = treated_weighted_mrs
results['change_in_weighted_mrs'] = change_in_weighted_mrs
results['untreated_utility'] = untreated_utility
results['treated_utility'] = treated_utility
results['change_in_utility'] = change_in_utility

results.to_csv('./output/thrombolysis_benefit_summary.csv')


In [8]:
results.round(3).T

,actually treated,improved_outcome including anticoagulant,improved_outcome_no_anticoagulant,benchmark
proportion_ischaemic_treated,0.352,0.496,0.433,0.432
untreated_mrs_0_1,0.300,0.229,0.239,0.294
treated_mrs_0_1,0.357,0.289,0.301,0.353
change_in_mrs_0_1,0.057,0.060,0.063,0.059
untreated_mrs_0_2,0.471,0.378,0.391,0.465
treated_mrs_0_2,0.539,0.457,0.471,0.534
change_in_mrs_0_2,0.068,0.079,0.080,0.069
untreated_mrs6,0.171,0.219,0.209,0.174
treated_mrs6,0.142,0.174,0.165,0.144
change_in_mrs6,-0.029,-0.044,-0.044,-0.030


In [9]:
# Additional good outcomes per 1000 arrivals (4 hours)

summary = pd.DataFrame()

summary['additional mrs 0-1 per 1000 arrivals'] = (
    results['proportion_ischaemic_treated'] * results['change_in_mrs_0_1'] * 1000)

summary['additional mrs 0-2 per 1000 arrivals'] = (
    results['proportion_ischaemic_treated'] * results['change_in_mrs_0_2'] * 1000)

summary['mean change in weighted mRS'] = (
    results['proportion_ischaemic_treated'] * results['change_in_weighted_mrs'])

summary['mean change in utility'] = (
    results['proportion_ischaemic_treated'] * results['change_in_utility'])

summary['mean change in death'] = (
    results['proportion_ischaemic_treated'] * results['change_in_mrs6'])

summary = summary.round(3)

In [10]:
summary.T

,actually treated,improved_outcome including anticoagulant,improved_outcome_no_anticoagulant,benchmark
additional mrs 0-1 per 1000 arrivals,19.907,29.805,27.124,25.344
additional mrs 0-2 per 1000 arrivals,23.930,39.015,34.771,29.957
mean change in weighted mRS,-0.104,-0.182,-0.161,-0.131
mean change in utility,0.021,0.036,0.032,0.026
mean change in death,-0.010,-0.022,-0.019,-0.013


In [11]:
# What proportion of patients on anticoagulants are treated with thrombolysis?

mask = (data['afib_anticoagulant'] == 1)
selected_data = data[mask]
proportion_treated = selected_data['thrombolysis'].mean()
print(f'Proportion of patients on anticoagulants treated with thrombolysis: {proportion_treated:0.4f}')
proportion_treated = selected_data['benchmark_decision'].mean()
print(f'Proportion of patients on anticoagulants with benchmark decision: {proportion_treated:0.4f}')


Proportion of patients on anticoagulants treated with thrombolysis: 0.0906
Proportion of patients on anticoagulants with benchmark decision: 0.0239


In [12]:
# Proportion of benchmark decision that are improved outcomes
mask = (data['benchmark_decision'] == 1)
selected_data = data[mask]
proportion_improved = selected_data['improved_outcome'].mean()
print(f'Proportion of benchmark decisions that are improved outcomes: {proportion_improved:0.4f}')


Proportion of benchmark decisions that are improved outcomes: 0.7093


In [13]:
# Proportion of actual thrombolysis that are improved outcomes
mask = (data['thrombolysis'] == 1)
selected_data = data[mask]
proportion_improved = selected_data['improved_outcome'].mean()
print(f'Proportion of actual thrombolysis that are improved outcomes: {proportion_improved:0.4f}')


Proportion of actual thrombolysis that are improved outcomes: 0.6957


In [14]:
# Proportion of improved outcomes (no anticoagulation) that are benchmark decisions
mask = (data['improved_outcome'] == 1) & (data['afib_anticoagulant'] == 0)
selected_data = data[mask]
proportion_benchmark = selected_data['benchmark_decision'].mean()
print(f'Proportion of improved outcomes (no anticoagulation) that are benchmark decisions: {proportion_benchmark:0.4f}')

Proportion of improved outcomes (no anticoagulation) that are benchmark decisions: 0.7007


In [15]:
# Proportion of improved outcomes (no anticoagulation) that are treated with thrombolysis
mask = (data['improved_outcome'] == 1) & (data['afib_anticoagulant'] == 0)
selected_data = data[mask]
proportion_treated = selected_data['thrombolysis'].mean()
print(f'Proportion of improved outcomes (no anticoagulation) treated with thrombolysis: {proportion_treated:0.4f}')

Proportion of improved outcomes (no anticoagulation) treated with thrombolysis: 0.5446


In [16]:
#data = pd.read_csv('../output/thrombolysis_choice_results.csv')
#data = data.drop(columns=['Unnamed: 0'])

mask = (data['thrombolysis'] == 0) & (data['improved_outcome'] == 0)
total = mask.sum()
print ("No thrombolysis, no improved outcome: ", total)
mask = (data['thrombolysis'] == 0) & (data['improved_outcome'] == 1)
total = mask.sum()
print ("No thrombolysis, improved outcome: ", total)
mask = (data['thrombolysis'] == 1) & (data['improved_outcome'] == 0)
total = mask.sum()
print ("Thrombolysis, no improved outcome: ", total)
mask = (data['thrombolysis'] == 1) & (data['improved_outcome'] == 1)
total = mask.sum()
print ("Thrombolysis, improved outcome: ", total)


No thrombolysis, no improved outcome:  44895
No thrombolysis, improved outcome:  28445
Thrombolysis, no improved outcome:  12096
Thrombolysis, improved outcome:  27660


In [17]:
mask = (data['benchmark_decision'] == 0) & (data['improved_outcome'] == 0)
total = mask.sum()
print ("No thrombolysis, no improved outcome: ", total)
mask = (data['benchmark_decision'] == 0) & (data['improved_outcome'] == 1)
total = mask.sum()
print ("No thrombolysis, improved outcome: ", total)
mask = (data['benchmark_decision'] == 1) & (data['improved_outcome'] == 0)
total = mask.sum()
print ("Thrombolysis, no improved outcome: ", total)
mask = (data['benchmark_decision'] == 1) & (data['improved_outcome'] == 1)
total = mask.sum()
print ("Thrombolysis, improved outcome: ", total)

No thrombolysis, no improved outcome:  42787
No thrombolysis, improved outcome:  21446
Thrombolysis, no improved outcome:  14204
Thrombolysis, improved outcome:  34659


In [18]:
sensitivity = data['TP'].sum() / (data['TP'].sum() + data['FN'].sum())
specificity = data['TN'].sum() / (data['TN'].sum() + data['FP'].sum())

print(f'Sensitivity: {sensitivity:0.4f}')
print(f'Specificity: {specificity:0.4f}')


Sensitivity: 0.4930
Specificity: 0.7878


In [19]:
cols = ['age', 'male', 'untreated_weighted_mrs', 'treated_weighted_mrs']

data_for_he = data[cols]
data_for_he['untreated_weighted_mrs'] = np.round(data_for_he['untreated_weighted_mrs'], 0).astype(int)
data_for_he['treated_weighted_mrs'] = np.round(data_for_he['treated_weighted_mrs'], 0).astype(int)
data_for_he.head()

/tmp/ipykernel_7930/843028029.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_for_he['untreated_weighted_mrs'] = np.round(data_for_he['untreated_weighted_mrs'], 0).astype(int)
/tmp/ipykernel_7930/843028029.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_for_he['treated_weighted_mrs'] = np.round(data_for_he['treated_weighted_mrs'], 0).astype(int)


,age,male,untreated_weighted_mrs,treated_weighted_mrs
0,77.5,0,3,2
1,92.5,1,3,4
2,87.5,1,2,2
3,82.5,1,2,2
5,92.5,0,3,4


In [20]:
# Place fixed parameters in this DataFrame:
df_fixed_params = pd.DataFrame()
df_fixed_params['mRS'] = pd.Series(get_fixed_params('mRS'))
df_fixed_params['Dichotomous'] = pd.Series(get_fixed_params('Dichotomous'))

# Display the DataFrame in this roundabout way to view wider columns
# and so not cut off lists:
with pd.option_context('display.max_colwidth', 200):
    display(df_fixed_params)


,mRS,Dichotomous
time_max_post_discharge_year,50,50
qaly_age_coeff,0.000259,0.000259
qaly_age2_coeff,0.000033,0.000033
qaly_sex_coeff,0.021213,0.021213
discount_factor_QALYs_perc,3.5,3.5
discount_factor_costs_perc,3.5,3.5
wtp_qaly_gpb,20000,20000
cost_ae_gbp,170.46,170.46
cost_elective_bed_day_gbp,443.8,443.8
cost_non_elective_bed_day_gbp,532.56,532.56


In [21]:
# Store patients in here:
patient_df = pd.DataFrame(columns=[
    'age', 'sex', 'sex_label', 'mrs'])

patient_df['age'] = data_for_he['age']
patient_df['sex'] = data_for_he['male']
f = lambda x: 'male' if x == 1 else 'female'
patient_df['sex_label'] = patient_df['sex'].map(f)
patient_df['untreated_mrs'] = data_for_he['untreated_weighted_mrs']
patient_df['treated_mrs'] = data_for_he['treated_weighted_mrs']

# Reindex the DataFrame:
patient_df = patient_df.reset_index(drop=True)

In [22]:
model_type_str = 'mRS'
fixed_params = get_fixed_params(model_type_str)

# Get the fixed parameters dictionary.
# This is found via a function because the parameters used
# depend on whether we're using the separate-mRS or the dichotomous
# model.
fixed_params = get_fixed_params(model_type_str)

# Store results dictionaries in here:
results_dict_list = []

# Run each patient separately:
for p in range(len(patient_df)):
    
    patient = patient_df.loc[p]

    if patient['untreated_mrs'] == 6:
        results_dict_list.append(dict())
        continue

    # This is the same code as in "2_Interactive_demo.py".
    results_dict = main_calculations(
        patient['age'],
        patient['sex'],
        patient['sex_label'],
        patient['untreated_mrs'],
        fixed_params,
        model_type_str)
    # Store this dictionary in the list of dicts:
    results_dict_list.append(results_dict)

# Turn all results dictionaries into a single data frame:
untreated_he = pd.DataFrame(results_dict_list)

In [23]:
patient

age              72.5
sex                 1
sex_label        male
mrs               NaN
untreated_mrs       3
treated_mrs         3
Name: 113095, dtype: object

In [24]:
untreated_he.head()

,age,sex,sex_label,model_type,mrs,outcome_type,death_in_year_1_lp,death_in_year_1_prob,death_in_year_n_lp,years,...,el_count,el_counts_by_year,el_discounted_by_year,el_discounted_cost,care_years,care_years_by_year,care_years_discounted_by_year,care_years_discounted_cost,total_discounted_cost,net_benefit
0,77.5,0.0,female,mRS,3.0,Dependent,-1.690997,0.155645,-7.962550,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",...,0.724209,"[0.24178915507340057, 0.2389491602628678, 0.20...","[0.24178915507340057, 0.230868753877167, 0.195...",310.153955,0.344997,"[0.10857142857142857, 0.10857142857142857, 0.1...","[0.10857142857142857, 0.10489993098688752, 0.1...",12454.474891,25313.194080,11934.509770
1,92.5,1.0,male,mRS,3.0,Dependent,-0.420518,0.396393,-6.470368,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",...,0.335961,"[0.29688300959223207, 0.039078106825869596]","[0.29688300959223207, 0.037756624952531015]",148.513070,0.122760,"[0.10857142857142857, 0.014188965044530086]","[0.10857142857142857, 0.01370914497056047]",4584.194764,13934.761210,-1624.259689
2,87.5,1.0,male,mRS,2.0,Independent,-1.729645,0.150633,-7.532428,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",...,1.191747,"[0.582176228808732, 0.4412244772707292, 0.1683...","[0.582176228808732, 0.42630384277365146, 0.157...",517.308099,0.021532,"[0.008689024390243901, 0.008689024390243901, 0...","[0.008689024390243901, 0.00839519264757865, 0....",785.857431,10801.264074,26098.209721
3,82.5,1.0,male,mRS,2.0,Independent,-2.061220,0.112924,-7.962456,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",...,1.456859,"[0.5613861903600795, 0.43169160588028954, 0.32...","[0.5613861903600795, 0.4170933390147725, 0.303...",624.444623,0.030520,"[0.008689024390243901, 0.008689024390243901, 0...","[0.008689024390243901, 0.00839519264757865, 0....",1095.117687,11979.082360,41252.855580
4,92.5,0.0,female,mRS,3.0,Dependent,-0.696270,0.332640,-6.557732,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",...,0.341138,"[0.2743295490722655, 0.06680884105649587]","[0.2743295490722655, 0.06454960488550326]",150.394569,0.134684,"[0.10857142857142857, 0.02611273856040197]","[0.10857142857142857, 0.02522969909217582]",5016.090545,15683.143260,-2747.781379


In [25]:
model_type_str = 'mRS'
fixed_params = get_fixed_params(model_type_str)

# Get the fixed parameters dictionary.
# This is found via a function because the parameters used
# depend on whether we're using the separate-mRS or the dichotomous
# model.
fixed_params = get_fixed_params(model_type_str)

# Store results dictionaries in here:
results_dict_list = []

# Run each patient separately:
for p in range(len(patient_df)):
    
    patient = patient_df.loc[p]

    if patient['treated_mrs'] == 6:
        results_dict_list.append(dict())
        continue

    # This is the same code as in "2_Interactive_demo.py".
    results_dict = main_calculations(
        patient['age'],
        patient['sex'],
        patient['sex_label'],
        patient['treated_mrs'],
        fixed_params,
        model_type_str        )
    # Store this dictionary in the list of dicts:
    results_dict_list.append(results_dict)

# Turn all results dictionaries into a single data frame:
treated_he = pd.DataFrame(results_dict_list)

In [26]:
treated_he.head()

,age,sex,sex_label,model_type,mrs,outcome_type,death_in_year_1_lp,death_in_year_1_prob,death_in_year_n_lp,years,...,el_count,el_counts_by_year,el_discounted_by_year,el_discounted_cost,care_years,care_years_by_year,care_years_discounted_by_year,care_years_discounted_cost,total_discounted_cost,net_benefit
0,77.5,0.0,female,mRS,2.0,Independent,-2.668548,0.064855,-8.469719,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",...,1.778422,"[0.5041964289700478, 0.4039097240586259, 0.310...","[0.5041964289700478, 0.39025094111944536, 0.28...",744.117405,0.048354,"[0.008689024390243901, 0.008689024390243901, 0...","[0.008689024390243901, 0.00839519264757865, 0....",1677.171007,14819.155063,66881.540998
1,92.5,1.0,male,mRS,4.0,Dependent,0.704697,0.669228,-6.230731,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",...,0.121007,[0.12100686701571335],[0.12100686701571335],53.702848,0.294469,[0.29446920314444586],[0.29446920314444586],11039.400127,18555.161216,-14063.691713
2,87.5,1.0,male,mRS,2.0,Independent,-1.729645,0.150633,-7.532428,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",...,1.191747,"[0.582176228808732, 0.4412244772707292, 0.1683...","[0.582176228808732, 0.42630384277365146, 0.157...",517.308099,0.021532,"[0.008689024390243901, 0.008689024390243901, 0...","[0.008689024390243901, 0.00839519264757865, 0....",785.857431,10801.264074,26098.209721
3,82.5,1.0,male,mRS,2.0,Independent,-2.061220,0.112924,-7.962456,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",...,1.456859,"[0.5613861903600795, 0.43169160588028954, 0.32...","[0.5613861903600795, 0.4170933390147725, 0.303...",624.444623,0.030520,"[0.008689024390243901, 0.008689024390243901, 0...","[0.008689024390243901, 0.00839519264757865, 0....",1095.117687,11979.082360,41252.855580
4,92.5,0.0,female,mRS,4.0,Dependent,0.428944,0.605622,-6.318096,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",...,0.134484,[0.1344842551839799],[0.1344842551839799],59.684112,0.350133,[0.3501329253148325],[0.3501329253148325],13126.185757,22233.342311,-17208.899472


In [27]:
list(treated_he)

['age',
 'sex',
 'sex_label',
 'model_type',
 'mrs',
 'outcome_type',
 'death_in_year_1_lp',
 'death_in_year_1_prob',
 'death_in_year_n_lp',
 'years',
 'hazard_by_year',
 'survival_by_year',
 'fhazard_by_year',
 'death_in_year_n_probs',
 'death_in_year_n_probs_first_invalid_index',
 'survival_median_years',
 'survival_lower_quartile_years',
 'survival_upper_quartile_years',
 'life_expectancy',
 'year_when_zero_survival',
 'qalys_total',
 'qalys_by_year',
 'raw_qalys_by_year',
 'ae_lp',
 'ae_count',
 'ae_counts_by_year',
 'ae_discounted_by_year',
 'ae_discounted_cost',
 'nel_lp',
 'nel_count',
 'nel_counts_by_year',
 'nel_discounted_by_year',
 'nel_discounted_cost',
 'el_lp',
 'el_count',
 'el_counts_by_year',
 'el_discounted_by_year',
 'el_discounted_cost',
 'care_years',
 'care_years_by_year',
 'care_years_discounted_by_year',
 'care_years_discounted_cost',
 'total_discounted_cost',
 'net_benefit']

In [28]:
data['used_untreated_mrs'] = data_for_he['untreated_weighted_mrs']
data['used_treated_mrs'] = data_for_he['treated_weighted_mrs']
data['untreated_survival_median_years'] = untreated_he['survival_median_years']
data['treated_survival_median_years'] = treated_he['survival_median_years']
data['untreated_life_expectancy'] = untreated_he['life_expectancy']
data['treated_life_expectancy'] = treated_he['life_expectancy']
data['untreated_qalys_total'] = untreated_he['qalys_total']
data['treated_qalys_total'] = treated_he['qalys_total']
data['untreated_care_years'] = untreated_he['care_years']
data['treated_care_years'] = treated_he['care_years']
data['untreated_total_discounted_cost'] = untreated_he['total_discounted_cost']
data['treated_total_discounted_cost'] = treated_he['total_discounted_cost']

In [29]:
# Cgange display rows to 100
pd.set_option('display.max_rows', 100)

data.head().T

,0,1,2,3,5
stroke_team,Royal Berkshire Hospital,Queen Elizabeth the Queen Mother Hospital,Royal Berkshire Hospital,Royal Berkshire Hospital,Queen Elizabeth Hospital Edgbaston
male,0,1,1,1,0
year,2017,2017,2017,2021,2016
arrive_by_ambulance,1.0,1.0,1.0,1.0,1.0
onset_to_arrival_time,65.0,110.0,190.0,90.0,111.0
onset_during_sleep,0,0,0,0,0
arrival_to_scan_time,5.0,75.0,15.0,16.0,39.0
onset_to_scan,70.0,185.0,205.0,106.0,150.0
infarction,1.0,1.0,1.0,1.0,1.0
stroke_severity,17,2,6,2,1


In [30]:
data.to_csv('./output/thrombolysis_choice_benefit_he.csv')